### 📘 Talleres de Ingeniería de Datos con Pyspark en Databricks 🐍🧱

---

👨‍💻 Autor: Brayan Neciosup  
📍 Portafolio: [brayanneciosup](https://bryanneciosup626.wixsite.com/brayandataanalitics)  
🔗 LinkedIn: [linkedin.com/brayanneciosup](https://www.linkedin.com/in/brayan-rafael-neciosup-bola%C3%B1os-407a59246/)  
💻 GitHub: [github.com/BrayanR03](https://github.com/BrayanR03)  
📚 Serie: Fundamentos de Apache Spark - PySpark 
📓 Estos talleres constarán de 3 niveles (Básico-Intermedio-Avanzado)   
🔍 Abarcará temas desde Fundamentos de Data Wrangling hacia Casos de Uso Avanzado   
📝 Cada ejercicio presenta su enunciado, dataset, resultado esperado y solución.   


#### FUNDAMENTOS DE DATA WRANGLING (MANIPULACIÓN DE DATOS)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("TalleresPySparkDE").getOrCreate()
### 💡 CABE RESALTAR QUE LOS DATASETS UTILIZADOS ESTAN ALMACENADOS EN UNITY CATALOG.

##### 🥉 NIVEL BÁSICO

In [0]:
"""
1. Detección de valores nulos en columnas principales

🗃️ Dataset: TITANIC
🗒️ Enunciado: Identifica cuántos valores faltantes hay en las columnas age, embarked y deck.
✍️ Resultado esperado: un conteo por columna con la cantidad de valores nulos.

"""
## ✔️ SOLUCIÓN
df_uno = spark.sql("SELECT * FROM workspace.exercises.titanic")
# df_uno.show(5)
cantidad_datos_nulos = df_uno.select([
  sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
  for c in df_uno.columns
])
cantidad_datos_nulos.show() ## ➡️ Cantidad de datos nulos: age(177) - embarked(2) - deck(688)


In [0]:
"""
2. Eliminación de filas duplicadas

🗃️ Dataset: Diccionario
🗒️ Enunciado: Elimina las filas duplicadas y conserva solo la primera aparición de cada registro.
✍️ Resultado esperado: un DataFrame sin filas repetidas.

"""
## ✔️ Solución
dict_data = {
 "id": [1,2,2,3,4,4,5],
 "nombre": ["Ana","Luis","Luis","María","Pedro","Pedro","Sofía"],
 "edad": [23,30,30,22,40,40,29]
}
df_dos = spark.createDataFrame(data=list(zip(*dict_data.values())),schema=list(dict_data.keys()))
# df_dos.show()
print(df_dos.count()) ## Cantidad de datos originales: 7
df_dos_clean = df_dos.drop_duplicates(subset=["id","nombre","edad"])
# df_dos_clean.show()
print(df_dos_clean.count()) ## Cantidad de datos después de eliminar nulos: 5

In [0]:
"""
3. Reemplazo simple de valores faltantes

🗃️ Dataset: PENGUINS
🗒️ Enunciado: Reemplaza los valores nulos en la columna bill_length_mm con la media de esa misma columna.
✍️ Resultado esperado: columna sin valores nulos en bill_length_mm.

"""
## ✔️ Solución
df_tres = spark.sql("SELECT * FROM workspace.exercises.penguins")
# df_tres.show(5)
cantidad_nulos_df_tres = df_tres.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_tres.columns
    if c=='bill_length_mm'
])
# cantidad_nulos_df_tres.show() ## Cantidad de nulos: 2
media_columna_bill_length_mm = df_tres.select(round(mean(col("bill_length_mm")),2)).collect()[0][0]
media_columna_bill_length_mm ## Valor de la media: 43.92
df_tres = df_tres.fillna(value=media_columna_bill_length_mm,subset=["bill_length_mm"])

cantidad_nulos_df_tres = df_tres.select([
    sum(when(col(c).isNull(),1).otherwise(0)).alias(c)
    for c in df_tres.columns
    if c=='bill_length_mm'
])
cantidad_nulos_df_tres.show() ## Cantidad de nulos: 0